# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [12]:
import logging
import ray
from random import randint

In [13]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2021-12-23_18-42-36_376073_42796/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-23_18-42-36_376073_42796/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-23_18-42-36_376073_42796',
 'metrics_export_port': 64464,
 'node_id': 'd9742b93601fab6d68d87bcd637d7ea11407c1b52abccb0715c02142'}

## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator:

Let's use Python class and convert that to a remote Actor class and create multiple actor handle instances associated with a distinct attributes, such as a name, age, goals scored, etc

In [14]:
@ray.remote
class GoalsScored:
    def __init__ (self, player, age) -> None:
        self._goals = 0
        self._player = player
        self._age = age

    def score (self, goal) -> object:
        self._goals += goal
        return self._goals
       
    def player(self) -> str:
        return self._player
    
    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=3)
    def stats(self) -> object:
        return self._player, self._age, self._goals

Define three Actors: Rolando, Neymar, Messi

In [15]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo", randint(18, 35))
neymar = GoalsScored.remote("Neymar", randint(18, 35))
messi = GoalsScored.remote("Messi", randint(18, 35))

CPU times: user 8.2 ms, sys: 3.1 ms, total: 11.3 ms
Wall time: 8.58 ms


Update the scores for each player

In [16]:
%%time

ronaldo.score.remote(randint(1, 7))
neymar.score.remote(randint(1, 7))
messi.score.remote(randint(1, 7))

CPU times: user 411 µs, sys: 85 µs, total: 496 µs
Wall time: 313 µs


ObjectRef(d56d800cbde6ca14a4890ed76268e44e7011b4ed0100000001000000)

Again, use list comprehension to iterate over each Actor handle instances, along with object_ref for their goals scores, maintained by each distinct actor.

In [17]:
def print_stats():
    for ref in [ronaldo, neymar, messi]:
        print(f"Player: {ray.get(ref.stats.remote())}")

In [18]:
print_stats()

Player: ['Ronaldo', 32, 2]
Player: ['Neymar', 27, 7]
Player: ['Messi', 21, 1]


Add three goals for for Neymar

In [21]:
[neymar.score.remote(goal) for goal in range(3)]
print_stats()

Player: ['Ronaldo', 32, 2]
Player: ['Neymar', 27, 16]
Player: ['Messi', 21, 1]


Finally, shutdown Ray

In [22]:
ray.shutdown()

---
## References

 * [A Universal Modular Actor Formalism for Artificial Intelligence](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  *IJCAI* (1973)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)